# Loading all of the data.

In [1]:
import pandas as pd
from sklearn import linear_model, model_selection, metrics

regions = ['eune', 'euw', 'na', 'kr', 'jp', 'tr', 'oce', 'br', 'lan', 'ru', 'las']

### Combining all the data from the different regions.

In [2]:
df = pd.DataFrame()

for region in regions:
    # Couldn't add the specific region csv files, since it's still collecting data on my pc.
    temp_df = pd.read_csv(f'data_{region}.csv')
    temp_df['region'] = region
    df = df.append(temp_df)

df.shape

(18308, 103)

In [3]:
df.head()

,match_id,1_kda,1_winrate,1_cm_points,1_cm_level,1_runes,1_role,1_champion,1_champion_winrate,1_champion_pickrate,...,10_cm_points,10_cm_level,10_runes,10_role,10_champion,10_champion_winrate,10_champion_pickrate,10_champion_banrate,won,region
0,2247671800,1.31,0.2,1080452,7,Arcane Comet,Role.solo,Lux,50.44,8.86,...,248084,7,Conqueror,Role.none,Lee Sin,47.46,11.51,1.34,1,eune
1,2247903911,3.55,1.0,266017,7,Press the Attack,Role.duo_carry,Draven,49.56,4.20,...,202116,7,Arcane Comet,Role.solo,Camille,47.97,3.40,0.19,1,eune
2,2247874642,3.83,0.5,361640,7,Arcane Comet,Role.solo,Xerath,50.85,3.28,...,111927,5,Conqueror,Role.solo,Aatrox,47.46,4.76,3.09,0,eune
3,2247565707,0.00,0.0,125787,7,Press the Attack,Role.duo_carry,Lucian,48.18,7.49,...,58676,7,Hail of Blades,Role.none,Rek'Sai,50.92,1.75,0.19,0,eune
4,2247891175,2.50,1.0,202116,7,Arcane Comet,Role.duo,Camille,47.97,3.40,...,751937,7,Electrocute,Role.duo,Lux,50.44,8.86,0.49,1,eune


In [4]:
# Save all of the data into one csv file for easier managing later on.
# Plus 'data.csv' is too large to upload to github enterprise, so uploading all files seperately is better.

df.to_csv('data.csv', index=None, header=True)

I have a few other features that I think would work, but I don't know how to apply correctly.
I think adding previous games of the player on a certain champion, or the current role would help with the prediction, with older matches having less weight than recent ones.

I can do this easily since I can query the api using timestamps that can be easily gotten from the match id.

In [3]:
features = list()
main_features = ['kda', 'winrate', 'cm_points', 'cm_level', 'champion_winrate', 'champion_pickrate', 'champion_banrate']

for i in range(1, 11):
    for feature in main_features:
        features.append(f'{i}_{feature}')

In [ ]:
X = df[features]
y = df['won']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=46)
logit_simple = linear_model.LogisticRegression(C=1e9).fit(X_train, y_train)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
